In [8]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import sys
import os
os.chdir('..')
sys.path.insert(0, os.getcwd())
import numpy as np

from AE_pipeline_pytorch import (
    LSTMAutoencoder,
    BiLSTMAutoencoder,
    
    GaitBatchIterable,

    train_autoencoder,
    evaluate_and_detect,
    extract_and_save_latents,
    reconstruct_and_evaluate,
    evaluate_autoencoder
)

import math
import torch
from time import time
from torch.utils.data import Dataset, DataLoader
import zarr
import torch.backends.cudnn as cudnn


2025-06-30 15:13:17.851549: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-30 15:13:17.858204: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751307197.866376 1569035 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751307197.868684 1569035 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751307197.874902 1569035 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Using device: cuda


In [2]:
print(os.getcwd())

/mnt/storage/dmartinez/Gait-Stability


In [3]:
#Data loader for Zarr datasets
batch_size = 400
train_loader = DataLoader(GaitBatchIterable("train_cycles.zarr",batch_size=batch_size, return_meta=False),
    batch_size=None,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=2,
)


val_loader = DataLoader(GaitBatchIterable("val_cycles.zarr",batch_size=batch_size, return_meta=False),
    batch_size=None,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=2,
)

test_loader = DataLoader(
    GaitBatchIterable("test_cycles.zarr", batch_size=batch_size, return_meta=True), 
    batch_size=None, 
    num_workers=8,   
    shuffle=False,   
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=2,
)


In [4]:
#To check the data loader
batch = next(iter(train_loader))
for i, item in enumerate(batch):
    print(f"Item {i}: type={type(item)}, shape={getattr(item, 'shape', None)}")

Item 0: type=<class 'torch.Tensor'>, shape=torch.Size([400, 100, 321])
Item 1: type=<class 'torch.Tensor'>, shape=torch.Size([400, 100, 321])


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cudnn.benchmark = True
print(f"Using device: {device}")

Using device: cuda


In [ ]:
#Train 

model = BiLSTMAutoencoder(n_timesteps=100, n_vars=321, latent_dim=128).to(device)


# Compila el modelo (PyTorch 2.0 JIT), **antes** de entrenar
if hasattr(torch, "compile"):
    model = torch.compile(model)


# Tu función de train_autoencoder acepta train_loader y val_loader
train_autoencoder(
    model,
    train_loader,
    val_loader,
    run_id="Torch1",
    epochs=30,
    debug=False,
    debug_batches=60
)

Epoch 1/30: train_loss=0.8804, val_loss=0.7183


/home/dmartinez/miniconda3/envs/gait-stability/lib/python3.10/site-packages/torch/utils/data/dataloader.py:718: UserWarning: Length of IterableDataset <AE_pipeline_pytorch.GaitBatchIterable object at 0x71f496b3fe50> was reported to be 486(when accessing len(dataloader)), but 487 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset for examples.
  warnings.warn(warn_msg)
/home/dmartinez/miniconda3/envs/gait-stability/lib/python3.10/site-packages/torch/utils/data/dataloader.py:718: UserWarning: Length of IterableDataset <AE_pipeline_pytorch.GaitBatchIterable object at 0x71f340efb250> was reported to be 105(when accessing len(dataloader)), but 106 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please

Epoch 2/30: train_loss=0.5848, val_loss=0.6337
Epoch 3/30: train_loss=0.5254, val_loss=0.6002
Epoch 4/30: train_loss=0.4929, val_loss=0.5782
Epoch 5/30: train_loss=0.4697, val_loss=0.5590
Epoch 6/30: train_loss=0.4515, val_loss=0.5452
Epoch 7/30: train_loss=0.4363, val_loss=0.5343
Epoch 8/30: train_loss=0.4237, val_loss=0.5255
Epoch 9/30: train_loss=0.4130, val_loss=0.5180
Epoch 10/30: train_loss=0.4037, val_loss=0.5106
Epoch 11/30: train_loss=0.3952, val_loss=0.5042
Epoch 12/30: train_loss=0.3876, val_loss=0.4987
Epoch 13/30: train_loss=0.3808, val_loss=0.4927
Epoch 14/30: train_loss=0.3751, val_loss=0.4866
Epoch 15/30: train_loss=0.3690, val_loss=0.4822


In [ ]:
#Evaluate metrics 
model = LSTMAutoencoder(n_timesteps=100, n_vars=321, latent_dim=128).to(device)
model.load_state_dict(torch.load("saved_models/ae_lstm_Torch1.pth"))
model.eval()



metrics = evaluate_autoencoder(model, train_loader, device=device)
print(metrics)



In [ ]:
model.eval()
mse = torch.nn.MSELoss(reduction='none')

all_scores = []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        recon = model(x)                       # (B, T, 321)
        # Si quieres comparar recon vs y, quizá tu función `evaluate_and_detect`
        # espera (x, y) y devuelve errores. Sino:
        # ejemplo de score: media de MSE por variable
        loss = mse(recon, x).mean(dim=(1,2))   # reconstrucción de x vs x
        # o si tu supervisión es recon vs y:
        # loss = mse(recon, y).mean(dim=(1,2))
        all_scores.append(loss.cpu())

all_scores = torch.cat(all_scores)
print("Test MSE (batch-wise):", all_scores.mean().item())
